# Elyse Hallstrom
# Data Science Certification Capston
# Phoenix Restaurant Location

# 0) Import Libraries and Dependencies

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 1) Get the Data Sources

### A) Zip code

In [3]:
zipcode = pd.read_excel(r'C:\Users\emhalls\Documents\Professional\Professional_Certifications\Data_Science\Capstone\us-zip-code-latitude-and-longitude.xlsx')
zipcode

### B) Average Household Income by Zip code

In [4]:
income_raw = pd.read_excel(r'C:\Users\emhalls\Documents\Professional\Professional_Certifications\Data_Science\Capstone\average_household_income_by_zip.xlsx')
income_raw

,#,Zip Code,Location,City,Population,Avg. Income/H/hold,National Rank
0,1,85045,"33.299769, -112.123507","Phoenix, Arizona",4558,105090,#193
1,2,85054,"33.683928, -111.949697","Phoenix, Arizona",2032,87106,#552
2,3,85048,"33.303055, -112.034014","Phoenix, Arizona",33431,82865,#695
3,4,85086,"33.846754, -112.085914","Phoenix, Arizona",8655,66636,"#1,924"
4,5,85028,"33.582359, -112.004940","Phoenix, Arizona",20565,65427,"#2,087"
5,6,85050,"33.704782, -111.990423","Phoenix, Arizona",19177,64971,"#2,154"
6,7,85044,"33.339569, -112.040475","Phoenix, Arizona",39892,60095,"#3,008"
7,8,85024,"33.683927, -112.025062","Phoenix, Arizona",19324,56463,"#3,763"
8,9,85085,"33.773409, -112.074522","Phoenix, Arizona",577,49375,"#6,170"
9,10,85037,"33.490164, -112.268027","Phoenix, Arizona",33150,49101,"#6,273"


### C) Get Set Up to Pull Foursquare Data

In [5]:
# Define FourSquare Credentials and Version
CLIENT_ID = 'CFFXEVBDYCRURZXQNXZB13AJLPACWIAQ41O40EH1EBXP3JL3' # your Foursquare ID
CLIENT_SECRET = 'XKR55TDBMY3DBPUZCW3ZSHLASUTA05RQUL2CMCFEYFSVQXNJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CFFXEVBDYCRURZXQNXZB13AJLPACWIAQ41O40EH1EBXP3JL3
CLIENT_SECRET:XKR55TDBMY3DBPUZCW3ZSHLASUTA05RQUL2CMCFEYFSVQXNJ


In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius

# 2) Clean the Data Sources

### A) Filter Zip Codes Down to Phoenix Zipcodes

In [8]:
az_zip = zipcode[zipcode['State'] == 'AZ'].reset_index(drop=True)
az_zip = az_zip[az_zip['City'] == 'Phoenix'].reset_index(drop=True)
az_zip = az_zip[['Zip', 'City', 'State', 'Latitude', 'Longitude']]
az_zip = az_zip.drop(columns=['State', 'City'])
az_zip

,Zip,Latitude,Longitude
0,85074,33.276539,-112.187170
1,85061,33.276539,-112.187170
2,85001,33.703967,-112.351835
3,85031,33.493496,-112.171080
4,85041,33.386995,-112.100760
5,85075,33.276539,-112.187170
6,85013,33.507110,-112.084830
7,85017,33.514092,-112.122850
8,85068,33.276539,-112.187170
9,85023,33.638271,-112.093410


### B) Clean Income Dataset
Select columns, break out state and city, break out longitude and latitude, rename columns, join with the zip code longitude and latitude

In [9]:
income = income_raw[['Zip Code', 'Location', 'City', 'Avg. Income/H/hold']]
income[['City','State']] = income['City'].str.split(pat=',', expand=True)
income = income.merge(az_zip, left_on='Zip Code', right_on='Zip')
income = income.rename(columns={'Avg. Income/H/hold': 'Avg Income'})
income = income.drop(columns=['Location'])
income

C:\Users\emhalls\Documents\Anaconda\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Zip Code,City,Avg Income,State,Zip,Latitude,Longitude
0,85045,Phoenix,105090,Arizona,85045,33.299740,-112.09812
1,85054,Phoenix,87106,Arizona,85054,33.689558,-111.96463
2,85048,Phoenix,82865,Arizona,85048,33.302098,-112.02477
3,85028,Phoenix,65427,Arizona,85028,33.585037,-112.00815
4,85050,Phoenix,64971,Arizona,85050,33.683603,-111.99271
5,85044,Phoenix,60095,Arizona,85044,33.333591,-111.99246
6,85024,Phoenix,56463,Arizona,85024,33.675564,-112.04717
7,85085,Phoenix,49375,Arizona,85085,33.777760,-112.07945
8,85037,Phoenix,49101,Arizona,85037,33.493006,-112.25539
9,85053,Phoenix,46422,Arizona,85053,33.629911,-112.13122


### C) Pull Foursquare Data for Phoenix Zip Codes

In [10]:
#run for each neighborhood
phoenix_venues = getNearbyVenues(names= income['Zip Code'],
                                   latitudes=income['Latitude'],
                                   longitudes=income['Longitude']
                                  )

85045
85054
85048
85028
85050
85044
85024
85085
85037
85053
85027
85023
85022
85032
85018
85020
85012
85029
85051
85016
85021
85033
85013
85031
85019
85035
85041
85040
85014
85043
85008
85015
85017
85009
85003
85006
85007
85004
85034


# 3) Explore the Data

In [11]:
phoenix_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,85045,33.29974,-112.09812,Pyramid Trailhead,33.303089,-112.100489,Trail
1,85045,33.29974,-112.09812,Trail 621 Trailhead,33.300319,-112.097176,Trail
2,85045,33.29974,-112.09812,Holiday Lights Phoenix,33.301110,-112.096340,Lighting Store
3,85045,33.29974,-112.09812,AZPoolExpert,33.302710,-112.099600,Home Service
4,85045,33.29974,-112.09812,Bursera Trail,33.303389,-112.099932,Trail


In [12]:
phoenix_venues.describe()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
count,435.000000,435.000000,435.000000,435.000000,435.000000
mean,85017.974713,33.500328,-112.075688,33.499910,-112.076173
std,14.305321,0.072075,0.052963,0.072021,0.052798
min,85003.000000,33.299740,-112.255390,33.297809,-112.256520
25%,85004.000000,33.451093,-112.093410,33.450170,-112.095594
50%,85013.000000,33.493496,-112.070570,33.494683,-112.073897
75%,85028.000000,33.511325,-112.048750,33.509845,-112.051371
max,85054.000000,33.698731,-111.964630,33.699728,-111.960866


In [13]:
phoenix_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
85003,24,24,24,24,24,24
85004,94,94,94,94,94,94
85006,22,22,22,22,22,22
85007,6,6,6,6,6,6
85008,12,12,12,12,12,12
85009,3,3,3,3,3,3
85012,22,22,22,22,22,22
85013,35,35,35,35,35,35
85014,14,14,14,14,14,14


In [14]:
phoenix_venues.groupby('Venue Category').count().sort_values(['Neighborhood'], ascending=False)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Pizza Place,18,18,18,18,18,18
Coffee Shop,16,16,16,16,16,16
Mexican Restaurant,15,15,15,15,15,15
American Restaurant,14,14,14,14,14,14
Convenience Store,14,14,14,14,14,14
Sandwich Place,13,13,13,13,13,13
Hotel,11,11,11,11,11,11
Fast Food Restaurant,8,8,8,8,8,8
Restaurant,8,8,8,8,8,8


In [15]:
# one hot encoding
# one hot encoding
phoenix_onehot = pd.get_dummies(phoenix_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
phoenix_onehot['Neighborhood'] = phoenix_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [phoenix_onehot.columns[-1]] + list(phoenix_onehot.columns[:-1])
phoenix_onehot = phoenix_onehot[fixed_columns]

phoenix_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Baggage Locker,Bakery,Bank,Bar,Baseball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Diner,Disc Golf,Discount Store,Donut Shop,Ethiopian Restaurant,Fast Food Restaurant,Flea Market,Food,Food Service,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lawyer,Light Rail Station,Lighting Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Park,Pawn Shop,Performing Arts Venue,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shopping Mall,Smoke Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sports Bar,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,85045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,85045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,85045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,85045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,85045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [16]:
phoenix_onehot.shape

(435, 158)

In [17]:
phoenix_grouped = phoenix_onehot.groupby('Neighborhood').mean().reset_index()
phoenix_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Baggage Locker,Bakery,Bank,Bar,Baseball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Diner,Disc Golf,Discount Store,Donut Shop,Ethiopian Restaurant,Fast Food Restaurant,Flea Market,Food,Food Service,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lawyer,Light Rail Station,Lighting Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Park,Pawn Shop,Performing Arts Venue,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shopping Mall,Smoke Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sports Bar,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,85003,0.000000,0.000000,0.000000,0.000000,0.00,0.125000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.0,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.041667,0.00,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.041667,0.000000,0.00,0.000000,0.000000
1,85004,0.000000,0.010638,0.000000,0.000000,0.00,0.074468,0.000000,0.010638,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010638,0.021277,0.00,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010638,0.010638,0.085106,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.000000,0.010638,0.000000,0.000000,0.0,0.000000,0.0

In [18]:
az_zip_venue = phoenix_grouped[['Neighborhood', 'American Restaurant']]
az_zip_venue.sort_values(['American Restaurant'], ascending=False).head(10)

,Neighborhood,American Restaurant
15,85021,0.250000
24,85033,0.200000
0,85003,0.125000
1,85004,0.074468
20,85028,0.071429
7,85013,0.028571
32,85050,0.000000
31,85045,0.000000
30,85044,0.000000
29,85043,0.000000


In [19]:
#add in the venue data to the income table
full_data = income.merge(az_zip_venue, left_on = 'Zip Code', right_on = 'Neighborhood')
full_data

,Zip Code,City,Avg Income,State,Zip,Latitude,Longitude,Neighborhood,American Restaurant
0,85045,Phoenix,105090,Arizona,85045,33.299740,-112.09812,85045,0.000000
1,85054,Phoenix,87106,Arizona,85054,33.689558,-111.96463,85054,0.000000
2,85028,Phoenix,65427,Arizona,85028,33.585037,-112.00815,85028,0.071429
3,85050,Phoenix,64971,Arizona,85050,33.683603,-111.99271,85050,0.000000
4,85044,Phoenix,60095,Arizona,85044,33.333591,-111.99246,85044,0.000000
5,85024,Phoenix,56463,Arizona,85024,33.675564,-112.04717,85024,0.000000
6,85037,Phoenix,49101,Arizona,85037,33.493006,-112.25539,85037,0.000000
7,85053,Phoenix,46422,Arizona,85053,33.629911,-112.13122,85053,0.000000
8,85027,Phoenix,44381,Arizona,85027,33.698731,-112.11429,85027,0.000000
9,85023,Phoenix,44177,Arizona,85023,33.638271,-112.09341,85023,0.000000


In [20]:
#see if there is a correlation between mean American restaurants and average household income
r = np.corrcoef(full_data['Avg Income'], full_data['American Restaurant'])
r

array([[ 1.        , -0.11286331],
       [-0.11286331,  1.        ]])

#### Figure out the most popular restaurants in the highest income zip code

In [27]:
money_zip = income[income['Avg Income'] == income['Avg Income'].max()]['Zip Code']
print(money_zip[0])
highest_income_zip = phoenix_grouped[phoenix_grouped['Neighborhood'] == money_zip[0]]
highest_income_zip
highest_income_venue = highest_income_zip.melt(id_vars=["Neighborhood"], 
        var_name="Venue", 
        value_name="Value")
sorted_highest_income_venue = highest_income_venue.sort_values('Value',ascending=False)
sorted_highest_income_venue

85045


,Neighborhood,Venue,Value
145,85045,Trail,0.500000
86,85045,Lawyer,0.166667
71,85045,Home Service,0.166667
88,85045,Lighting Store,0.166667
0,85045,ATM,0.000000
108,85045,Pet Store,0.000000
101,85045,New American Restaurant,0.000000
102,85045,Nightclub,0.000000
103,85045,Noodle House,0.000000
104,85045,Opera House,0.000000


In [22]:
#figure out the zip code with the most american restaurants on average
rest_zip = az_zip_venue[az_zip_venue['American Restaurant'] == az_zip_venue['American Restaurant'].max()]['Neighborhood']
print(rest_zip[15])
highest_venue_zip = phoenix_grouped[phoenix_grouped['Neighborhood'] == rest_zip[15]]
highest_venue_zip

85021


,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Baggage Locker,Bakery,Bank,Bar,Baseball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Diner,Disc Golf,Discount Store,Donut Shop,Ethiopian Restaurant,Fast Food Restaurant,Flea Market,Food,Food Service,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lawyer,Light Rail Station,Lighting Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Park,Pawn Shop,Performing Arts Venue,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shopping Mall,Smoke Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sports Bar,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
15,85021,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
listed_highest_venue = highest_venue_zip.melt(id_vars=["Neighborhood"], 
        var_name="Venue", 
        value_name="Value")
sorted_listed_highest_venue = listed_highest_venue.sort_values('Value',ascending=False)
sorted_listed_highest_venue

,Neighborhood,Venue,Value
105,85021,Park,0.50
5,85021,American Restaurant,0.25
112,85021,Playground,0.25
0,85021,ATM,0.00
108,85021,Pet Store,0.00
101,85021,New American Restaurant,0.00
102,85021,Nightclub,0.00
103,85021,Noodle House,0.00
104,85021,Opera House,0.00
106,85021,Pawn Shop,0.00
